In [36]:
from Bio import SeqIO, Entrez
import os


In [37]:
def affichage(sequence,seg_list):
    print(f"id :{sequence.id}")
    print(f"longeur : {len(sequence.seq)}")
    seg_list.append(sequence.seq)
    print(f"séquence : {sequence.seq}")

In [38]:
def explore_alignments(i, j, current_align1, current_align2,alignments,score_matrix,seq1,seq2):
    # Condition d'arrêt
    if i < 2 and j < 2 :
        alignments.append((current_align1, current_align2))
        return

    diag = score_matrix[i - 1][j - 1] if i > 0 and j > 0 else float('-inf')
    up = score_matrix[i - 1][j] if i > 0 else float('-inf')
    left = score_matrix[i][j - 1] if j > 0 else float('-inf')

    max_value = max(diag, up, left)
    if max_value == diag:
        if i>1 and j>1 :
        
            explore_alignments(i - 1, j - 1, seq1[i -2] + current_align1, seq2[j -2] + current_align2,alignments,score_matrix,seq1,seq2)
        elif j<1 : 
            explore_alignments(i - 1, j - 1, seq1[i -2] + current_align1, '-' + current_align2,alignments,score_matrix,seq1,seq2)
        elif i<1:
             explore_alignments(i - 1, j - 1, '-' + current_align1, seq2[j -2] + current_align2,alignments,score_matrix,seq1,seq2)

    if max_value == up:
        explore_alignments(i - 1, j, seq1[i -2] + current_align1, '-' + current_align2,alignments,score_matrix,seq1,seq2)
    if max_value == left:
        explore_alignments(i, j - 1, '-' + current_align1, seq2[j -2] + current_align2,alignments,score_matrix,seq1,seq2)

    return alignments    


In [39]:
def global_alignment(seq1, seq2, match=2, mismatch=-1, gap=-1):

    n = len(seq1) + 1
    m = len(seq2) + 1

    score_matrix = [[0] * m for _ in range(n)]
    print(score_matrix)
    # traceback_matrix = [[None] * m for _ in range(n)]

    # Rempli gap
    for i in range(1, n):
        score_matrix[i][0] = score_matrix[i - 1][0] + gap
        # traceback_matrix[i][0] = 'up'
    for j in range(1, m):
        score_matrix[0][j] = score_matrix[0][j - 1] + gap
        # traceback_matrix[0][j] = 'left'


    for i in range(1, n):
        for j in range(1, m):
            if seq1[i - 1] == seq2[j - 1]:
                score = match
            else:
                score = mismatch

            #best score
            diag = score_matrix[i - 1][j - 1] + score
            up = score_matrix[i - 1][j] + gap
            left = score_matrix[i][j - 1] + gap
            max_score = max(diag, up, left)
            score_matrix[i][j] = max_score

            # if max_score == diag:
            #     traceback_matrix[i][j] = 'diag'
            # elif max_score == up:
            #     traceback_matrix[i][j] = 'up'
            # else:
            #     traceback_matrix[i][j] = 'left'

    alignments = []  
    i, j = n - 1, m - 1
    initial_align1 = seq1[i-1]  
    initial_align2 = seq2[j-1]  

    alignments = explore_alignments(i, j, initial_align1, initial_align2,alignments,score_matrix,seq1,seq2)
                    
    print("Matrice de score :")
    for row in score_matrix:
        print(row)    
    
    return alignments, score_matrix[n - 1][m - 1]




In [40]:
def local_alignment(seq1, seq2, match, mismatch, gap):
    n = len(seq1) + 1
    m = len(seq2) + 1

    score_matrix = [[0] * m for _ in range(n)]

    # max_pos = (0, 0)
    
    for i in range(1, n):
        for j in range(1, m):
            if seq1[i - 1] == seq2[j - 1]:
                score = match
            else:
                score = mismatch

            diag = score_matrix[i - 1][j - 1] + score
            up = score_matrix[i - 1][j] + gap
            left = score_matrix[i][j - 1] + gap
            
            score_matrix[i][j] = max(0,diag, up, left)
            
            # # Track maximum score position
            # if score_matrix[i][j] > max_score:
            #     max_score = score_matrix[i][j]

    End = score_matrix[n - 1][m - 1]
    diag = score_matrix[n - 2][m - 2]
    up = score_matrix[n - 2][m-1]
    left = score_matrix[n-1][m - 2]
    score_final = max(diag,up,left,End) 

    if  score_final == End:
        i, j = n - 1, m - 1
        initial_align1 = seq1[i-1]  
        initial_align2 = seq2[j-1]
    elif   score_final == diag:
        i, j = n - 2, m - 1
        initial_align1 = seq1[i-2]  
        initial_align2 = seq2[j-2]        
    elif   score_final == up:
        i, j = n - 2, m - 1
        initial_align1 = seq1[i-2]  
        initial_align2 = seq2[j-1]
    elif score_final == left: 
        i, j = n - 1, m - 2 
        initial_align1 = seq1[i-1]  
        initial_align2 = seq2[j-2]   


    alignments = []  
      

    alignments = explore_alignments(i, j, initial_align1, initial_align2,alignments,score_matrix,seq1,seq2)
       
    print("Matrice de score :")
    for row in score_matrix:
        print(row)
    
    return alignments, score_final


In [41]:
def calculate_score(align1, align2, match, mismatch, gap):
    score = 0
    for a, b in zip(align1, align2):
        if a == '-' or b == '-':
            score += gap
        elif a == b:
            score += match
        else:
            score += mismatch
    return score

In [42]:
# Exemple d'utilisation
seq1 = "CTTG"
seq2 = "ACTGG"
alignments, score = global_alignment(seq1, seq2,2,-1,-1)
print("Score de la matrice :", score)
print("Alignement de la séquence 1 :", alignments)
scored_alignments = [(align1, align2, f'score = {calculate_score(align1, align2,2,-1,-1)}' ) for align1, align2 in alignments]
print(scored_alignments)


[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
Matrice de score :
[0, -1, -2, -3, -4, -5]
[-1, -1, 1, 0, -1, -2]
[-2, -2, 0, 3, 2, 1]
[-3, -3, -1, 2, 2, 1]
[-4, -4, -2, 1, 4, 4]
Score de la matrice : 4
Alignement de la séquence 1 : [('-CTT-G', 'AC-TGG'), ('-CTT-G', 'ACT-GG')]
[('-CTT-G', 'AC-TGG', 'score = 3'), ('-CTT-G', 'ACT-GG', 'score = 3')]


In [43]:
alignments, score = local_alignment(seq1, seq2,2,-1,-2)
print("Score de la matrice :", score)
print("Alignement de la séquence 1 :", alignments)
scored_alignments = [(align1, align2, f'score = {calculate_score(align1, align2,2,-1,-2)}' ) for align1, align2 in alignments]
print(scored_alignments)


Matrice de score :
[0, 0, 0, 0, 0, 0]
[0, 0, 2, 0, 0, 0]
[0, 0, 0, 4, 2, 0]
[0, 0, 0, 2, 3, 1]
[0, 0, 0, 0, 4, 5]
Score de la matrice : 5
Alignement de la séquence 1 : [('-GCTT-G', 'A-CT-GG'), ('-CTT-G', 'ACT-GG')]
[('-GCTT-G', 'A-CT-GG', 'score = -2'), ('-CTT-G', 'ACT-GG', 'score = 0')]


In [49]:
import numpy as np
from Bio import pairwise2  # Pour les alignements globaux et locaux

class SequenceAligner:
    def global_alignment(self, seq1, seq2):
        # Utilisation de BioPython pour l'alignement global
        alignments = pairwise2.align.globalxx(seq1, seq2)
        best_alignment = alignments[0]
        score = best_alignment.score
        return best_alignment, score, alignments

    def local_alignment(self, seq1, seq2):
        # Utilisation de BioPython pour l'alignement local
        alignments = pairwise2.align.localxx(seq1, seq2)
        best_alignment = alignments[0]
        score = best_alignment.score
        return best_alignment, score, alignments

    def pair_par_pair(self, sequences, method='global'):
        n = len(sequences)
        alignment_scores = [[0] * n for _ in range(n)]

        for i in range(n):
            for j in range(i + 1, n):
                if method == 'global':
                    alignement, score, _ = self.global_alignment(sequences[i], sequences[j])
                    print(f"Alignement entre {sequences[i]} et {sequences[j]}:\n{alignement}")
                elif method == 'local':
                    _, score, _ = self.local_alignment(sequences[i], sequences[j])
                alignment_scores[i][j] = score
                alignment_scores[j][i] = score

        print("\nMatrice d'alignement :")
        for row in alignment_scores:
            print(row)
        return alignment_scores

    def construct_guide_tree(self, distance_matrix):
        num_seqs = len(distance_matrix)
        clusters = {i: [i] for i in range(num_seqs)}
        guide_tree = []

        while len(clusters) > 1:
            # Trouver les deux clusters les plus proches
            min_dist = float('inf')
            closest_pair = None

            for i in clusters:
                for j in clusters:
                    if i < j:
                        dist = np.mean([distance_matrix[x][y] for x in clusters[i] for y in clusters[j]])
                        if dist < min_dist:
                            min_dist = dist
                            closest_pair = (i, j)

            # Fusionner les deux clusters les plus proches
            i, j = closest_pair
            new_cluster = clusters[i] + clusters[j]
            guide_tree.append((clusters[i], clusters[j]))

            # Supprimer les anciens clusters
            del clusters[i]
            del clusters[j]

            # Ajouter le nouveau cluster
            new_cluster_id = max(clusters.keys(), default=-1) + 1
            clusters[new_cluster_id] = new_cluster

        return guide_tree

    def progressive_alignment(self, sequences, guide_tree):
        # Initialize the aligned_sequences with the original sequences
        aligned_sequences = [sequences[i] for i in range(len(sequences))]

        for cluster1, cluster2 in guide_tree:
            # Combine the sequences in cluster1 and cluster2
            seq1 = ''.join([aligned_sequences[i] for i in cluster1])
            seq2 = ''.join([aligned_sequences[i] for i in cluster2])
            
            # Perform global alignment
            alignment, score, _ = self.global_alignment(seq1, seq2)
            print(alignment)

            # Update aligned_sequences based on alignment
            aligned_seqA = alignment.seqA
            aligned_seqB = alignment.seqB

            # Update sequences for cluster1
            for i, seq in zip(cluster1, aligned_seqA):
                aligned_sequences[i] = seq

            # Update sequences for cluster2
            for i, seq in zip(cluster2, aligned_seqB):
                aligned_sequences[i] = seq

        return alignment



In [50]:
sequences = ["ACTGG", "ACTTGG", "ACTGC", "CTTG"]  
aligner = SequenceAligner()

# Étape 1 : Calcul des scores d'alignement par paires
scores = aligner.pair_par_pair(sequences)

print(scores)

# Étape 2 : Construction de l'arbre guide
guide_tree = aligner.construct_guide_tree(scores)

print(guide_tree)

# Étape 3 : Alignement progressif
aligned_sequences = aligner.progressive_alignment(sequences, guide_tree)
print("\nAlignements finaux :")
for seq in aligned_sequences:
    print(seq)


Alignement entre ACTGG et ACTTGG:
Alignment(seqA='ACT-GG', seqB='ACTTGG', score=5.0, start=0, end=6)
Alignement entre ACTGG et ACTGC:
Alignment(seqA='ACTGG-', seqB='ACT-GC', score=4.0, start=0, end=6)
Alignement entre ACTGG et CTTG:
Alignment(seqA='ACTG-G', seqB='-CT-TG', score=3.0, start=0, end=6)
Alignement entre ACTTGG et ACTGC:
Alignment(seqA='ACTTGG-', seqB='AC-T-GC', score=4.0, start=0, end=7)
Alignement entre ACTTGG et CTTG:
Alignment(seqA='ACTTGG', seqB='-CTT-G', score=4.0, start=0, end=6)
Alignement entre ACTGC et CTTG:
Alignment(seqA='ACT-GC', seqB='-CTTG-', score=3.0, start=0, end=6)

Matrice d'alignement :
[0, 5.0, 4.0, 3.0]
[5.0, 0, 4.0, 4.0]
[4.0, 4.0, 0, 3.0]
[3.0, 4.0, 3.0, 0]
[[0, 5.0, 4.0, 3.0], [5.0, 0, 4.0, 4.0], [4.0, 4.0, 0, 3.0], [3.0, 4.0, 3.0, 0]]
[([0], [3]), ([2], [0, 3]), ([1], [2, 0, 3])]
Alignment(seqA='ACTG-G', seqB='-CT-TG', score=3.0, start=0, end=6)
Alignment(seqA='ACTGC-', seqB='A-----', score=1.0, start=0, end=6)
Alignment(seqA='ACTTGG--', seqB='A---

In [5]:
import numpy as np

# Exemple de matrice de distances initiale
dist_matrix = np.array([
    [0, 2, 4, 2],
    [2, 0, 4, 4],
    [4, 4, 0, 6],
    [2, 4, 6, 0]
])

labels = ["I1", "I2", "I3", "I4"]

# Fonction pour afficher la matrice de distance
def print_distance_matrix(matrix, labels, step):
    print(f"\nÉtape {step} : Matrice de distances")
    print("   ", "  ".join(labels))
    for i, row in enumerate(matrix):
        print(f"{labels[i]:<3} {row}")

# Calcul de R_i pour chaque ligne
def calculate_Ri(matrix):
    n = len(matrix)
    Ri = np.sum(matrix, axis=1) / (n - 2)
    return Ri

# Calcul de S_ij (distance modifiée)
def calculate_Sij(matrix, Ri):
    n = len(matrix)
    Sij = np.zeros_like(matrix)
    for i in range(n):
        for j in range(n):
            if i != j:
                Sij[i][j] = matrix[i][j] - Ri[i] - Ri[j]
    return Sij

# Mise à jour de la matrice après regroupement
def update_distance_matrix(matrix, cluster_a, cluster_b):
    n = len(matrix)
    new_matrix = []
    for i in range(n):
        if i != cluster_a and i != cluster_b:
            new_row = []
            for j in range(n):
                if j != cluster_a and j != cluster_b:
                    new_row.append(matrix[i][j])
            new_matrix.append(new_row)
    # Calcul des nouvelles distances
    new_cluster_distances = []
    for i in range(n):
        if i not in (cluster_a, cluster_b):
            dist = (matrix[i][cluster_a] + matrix[i][cluster_b]) / 2
            new_cluster_distances.append(dist)
    new_cluster_distances.append(0)  # Distance à lui-même
    for i, row in enumerate(new_matrix):
        row.append(new_cluster_distances[i])
    new_matrix.append(new_cluster_distances)
    return np.array(new_matrix)

# Étapes de regroupement
step = 1
while len(dist_matrix) > 1:
    print_distance_matrix(dist_matrix, labels, step)
    
    # Calcul de R_i
    Ri = calculate_Ri(dist_matrix)
    print(f"\nÉtape {step} : Calcul des R_i : {Ri}")
    
    # Calcul de S_ij
    Sij = calculate_Sij(dist_matrix, Ri)
    print(f"\nÉtape {step} : Calcul de S_ij (distance modifiée):\n{Sij}")
    
    # Trouver les deux clusters les plus proches (min Sij)
    min_value = np.inf
    cluster_a, cluster_b = -1, -1
    for i in range(len(Sij)):
        for j in range(i + 1, len(Sij)):
            if Sij[i][j] < min_value:
                min_value = Sij[i][j]
                cluster_a, cluster_b = i, j

    print(f"\nÉtape {step} : Regrouper {labels[cluster_a]} et {labels[cluster_b]} (Sij = {min_value})")
    
    # Calcul des nouvelles distances d'arbres
    d_a = (dist_matrix[cluster_a, cluster_b] + Ri[cluster_a] - Ri[cluster_b]) / 2
    d_b = (dist_matrix[cluster_a, cluster_b] + Ri[cluster_b] - Ri[cluster_a]) / 2
    print(f"Distance {labels[cluster_a]} -> arbre = {d_a}")
    print(f"Distance {labels[cluster_b]} -> arbre = {d_b}")
    
    # Mise à jour des labels et de la matrice
    new_label = f"({labels[cluster_a]},{labels[cluster_b]})"
    labels = [label for i, label in enumerate(labels) if i not in (cluster_a, cluster_b)] + [new_label]
    dist_matrix = update_distance_matrix(dist_matrix, cluster_a, cluster_b)
    
    step += 1

print("\nClustering terminé!")



Étape 1 : Matrice de distances
    I1  I2  I3  I4
I1  [0 2 4 2]
I2  [2 0 4 4]
I3  [4 4 0 6]
I4  [2 4 6 0]

Étape 1 : Calcul des R_i : [4. 5. 7. 6.]

Étape 1 : Calcul de S_ij (distance modifiée):
[[ 0 -7 -7 -8]
 [-7  0 -8 -7]
 [-7 -8  0 -7]
 [-8 -7 -7  0]]

Étape 1 : Regrouper I1 et I4 (Sij = -8)
Distance I1 -> arbre = 0.0
Distance I4 -> arbre = 2.0

Étape 2 : Matrice de distances
    I2  I3  (I1,I4)
I2  [0. 4. 3.]
I3  [4. 0. 5.]
(I1,I4) [3. 5. 0.]

Étape 2 : Calcul des R_i : [7. 9. 8.]

Étape 2 : Calcul de S_ij (distance modifiée):
[[  0. -12. -12.]
 [-12.   0. -12.]
 [-12. -12.   0.]]

Étape 2 : Regrouper I2 et I3 (Sij = -12.0)
Distance I2 -> arbre = 1.0
Distance I3 -> arbre = 3.0

Étape 3 : Matrice de distances
    (I1,I4)  (I2,I3)
(I1,I4) [0. 4.]
(I2,I3) [4. 0.]

Étape 3 : Calcul des R_i : [inf inf]

Étape 3 : Calcul de S_ij (distance modifiée):
[[  0. -inf]
 [-inf   0.]]

Étape 3 : Regrouper (I1,I4) et (I2,I3) (Sij = -inf)
Distance (I1,I4) -> arbre = nan
Distance (I2,I3) -> arbre 

C:\Users\meriem\AppData\Local\Temp\ipykernel_2276\968301348.py:23: RuntimeWarning: divide by zero encountered in divide
  Ri = np.sum(matrix, axis=1) / (n - 2)
C:\Users\meriem\AppData\Local\Temp\ipykernel_2276\968301348.py:84: RuntimeWarning: invalid value encountered in scalar subtract
  d_a = (dist_matrix[cluster_a, cluster_b] + Ri[cluster_a] - Ri[cluster_b]) / 2
C:\Users\meriem\AppData\Local\Temp\ipykernel_2276\968301348.py:85: RuntimeWarning: invalid value encountered in scalar subtract
  d_b = (dist_matrix[cluster_a, cluster_b] + Ri[cluster_b] - Ri[cluster_a]) / 2
